In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from muxcnn.utils import load_params, load_img
#from muxcnn.hecnn_par import *
#from muxcnn.comparator import ApprRelu
import torch
from muxcnn.resnet_muxconv import ResNet_MuxConv

## Torch model

In [5]:
from muxcnn.models.ResNet20 import ResNet, BasicBlock

model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

In [6]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

In [7]:
print(model(img_tensor))

before softmax tensor([[ -9.3005, -11.0586,  -2.9741,  -2.9179,  -9.3500,   4.1474,  -6.8721,
          -3.6999,  -8.9446,  -7.6901]], grad_fn=<AddmmBackward0>)
tensor([[-1.3450e+01, -1.5208e+01, -7.1236e+00, -7.0674e+00, -1.3500e+01,
         -2.0791e-03, -1.1022e+01, -7.8494e+00, -1.3094e+01, -1.1840e+01]],
       grad_fn=<LogSoftmaxBackward0>)


# FHE

In [8]:
from time import time

In [9]:
fhe_model = ResNet_MuxConv(model, alpha=12)

functions set
degrees = [15, 15, 15, 15], margin = 0.01, eps = 0.02


In [24]:
t0 = time()
ctxt, outs0 = fhe_model.forward_early(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
2.4993 s
2.5166 s


In [25]:
print(ctxt[::1000])

[ 1.23083757e+00 -1.26932767e-09  8.31757006e-01  5.57055145e-01
  1.46344638e+00  5.52875764e-01  9.16643332e-10  3.22529485e-01
  1.66880482e-01 -1.47863135e-10 -4.83194805e-10  5.25957644e-02
 -1.02005576e-08  8.14742655e-01  3.52006406e-01  9.86855254e-01
 -5.28150661e-09 -2.99751585e-10  2.00753382e-01  4.62600850e+00
  1.45841416e+00  1.12151311e-01  2.53264001e-01  2.61802319e+00
  6.75758605e-02  2.28868991e-02 -9.50749179e-09  1.08192038e-09
  3.17374896e-09  9.83460247e-01  9.61286445e-01  4.02652655e-01
  3.70484902e-10]


In [17]:
ctxt2, outs1 = fhe_model.forward_bb(model.layer1[0], ctxt, outs0)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
15.1050 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
13.0218 s


In [22]:
print(ctxt2[::1000])

[ 2.29267736e+00  1.55785321e-01  2.07220434e-01  6.34568482e-01
  1.73384565e+00  4.63323207e-10  9.20647750e-01  1.24998424e-01
  7.27318138e-01  1.15989112e-09  4.58737551e-01  9.81055376e-01
 -7.62398747e-09  1.77523743e+00  1.86425350e-01  8.93213558e-01
  9.10392226e-01  4.85560791e-04  1.02342889e+00  1.03987971e+00
  1.41272675e+00  5.91650931e-10  1.30377909e-01  1.31060697e+00
  1.66285891e-01 -1.18811039e-09 -5.08807410e-09  1.49653335e-09
 -4.20501259e-09  7.86966889e-01  1.64648186e+00 -5.06177593e-09
  8.46363515e-01]


In [ ]:
ctxt2, outs1 = fhe_model.forward_bb(model.layer1[0], ctxt2, outs1)

In [13]:
print(result[:10])

(array([ 1.23083757e+00,  4.86967574e-01,  2.75344729e-01, ...,
       -5.07707003e-10,  7.09786031e-10,  1.15663971e-10]), {'h': 32, 'w': 32, 'c': 16, 'k': 1, 't': 16, 'p': 2})


In [20]:
t0 = time()
result = fhe_model(img_tensor)
print(f"{time() - t0:.4f} s")

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
1.9421 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5915 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 8
7.5881 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
14.5817 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 8
6.9168 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
2.1417 s
[MultParConv] (hi,wi,ci,ki,ti,pi) =(16,16,32, 2, 8,  4)
[MultParConv] (ho,wo,co,ko,to,po) =( 8, 8,64, 4, 4,  8)
[MultParConv] q = 16
13.3325 s
[MultPar

In [21]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)

In [10]:
torch.tensor(result[::64][:10])

tensor([ -9.2678, -10.8150,  -3.1036,  -2.9026,  -9.4616,   4.3280,  -6.9752,
         -3.5619,  -8.8842,  -7.5630], dtype=torch.float64)